In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Data Loading and Preprocessing

In [2]:
hr_df= pd.read_csv('data/turnover.csv')

In [3]:
hr_df = hr_df.rename(columns={'average_montly_hours':'average_monthly_hours', 'sales': 'role', 'Work_accident':'work_accident'})
hr_df.head()

,satisfaction_level,last_evaluation,number_project,average_monthly_hours,time_spend_company,work_accident,left,promotion_last_5years,role,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


#### Splitting the dataset

In [4]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(hr_df, test_size=0.2, random_state=289, stratify=hr_df.left)

#### Outliers removal

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
class OutlierRemoval(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self,X):
        return self
    
    def transform(self, X):
        for column in self.columns:
            q25, q75 = np.percentile(X[[column]], [25,75])
            iqr = q75-q25
            
            X = X[(X[column] <= (q75 + 1.5*iqr)) & (X[column] >= (q25 - 1.5*iqr))]
            
        return X
    
outlier_removal = OutlierRemoval(['last_evaluation', 'time_spend_company'])
train_set_adv = outlier_removal.fit_transform(train_set)

In [18]:
X = train_set.drop('left', axis=1).copy()
y = train_set['left'].copy()

X_adv = train_set_adv.drop('left', axis=1).copy()
y_adv = train_set_adv['left'].copy()

X_test = test_set.drop('left', axis=1).copy()
y_test = test_set['left'].copy()

In [19]:
X.head()

,satisfaction_level,last_evaluation,number_project,average_monthly_hours,time_spend_company,work_accident,promotion_last_5years,role,salary
8953,0.71,0.91,3,261,3,0,0,sales,low
4179,0.95,0.58,3,163,3,0,0,technical,medium
5672,0.77,0.80,4,167,3,0,0,sales,medium
955,0.10,0.91,6,262,5,1,0,technical,low
2170,0.81,0.94,3,225,4,0,0,hr,medium


#### Preprocessing 

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

class CustomOrdinalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, order={}):
        self.order = order
        
    def fit(self,X):
        return self
    
    def transform(self,X):
        if self.order :
            for column in X.columns:
                X.loc[:, column] =  X.loc[:, column].replace(list(self.order.keys()), list(self.order.values()))
    
        return X

binning_features = ['satisfaction_level', 'last_evaluation', 'average_monthly_hours']

basic_featuring = ColumnTransformer([
    ('role_enc', OneHotEncoder(), ['role']),
    ('binning', KBinsDiscretizer(n_bins=7, encode='ordinal'), binning_features),
    ('scaling', MinMaxScaler(), ['average_monthly_hours']),
    ('salary_enc', CustomOrdinalEncoder())
])



basic_featuring = ColumnTransformer([
    ('role_basic', OneHotEncoder(), ['role'])
], remainder='passthrough')


basic_feature_pipeline = Pipeline([
    ('basic_salary', CustomOrdinalEncoder('salary', {'low': 1, 'medium': 2, 'high': 3})),
    ('basic_role', basic_featuring),
    ('basic_scaling', MinMaxScaler())
])


advanced_featuring = ColumnTransformer([
    ('binning', KBinsDiscretizer(n_bins=7, encode='ordinal'), ),
    ('role_advanced', OneHotEncoder(), ['role'])
], remainder='passthrough')

advanced_feature_pipeline = Pipeline([
    ('advanced_salary', CustomOrdinalEncoder('salary', {'low':1, 'medium': 2, 'high': 3})),
    ('advanced_featuring', advanced_featuring),
    ('advanced_scaling', MinMaxScaler()),
    ('linear_reg', De1)
])


In [17]:
X_prepared = basic_feature_pipeline.fit_transform(X)
X_test_prepared = basic_feature_pipeline.transform(X_test)

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [9]:
X_adv_prepared = advanced_feature_pipeline.fit_transform(X_adv)
X_test_adv_prepared = advanced_feature_pipeline.transform(X_test)

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

#### Feature Engineering - nonregularized logit, all features

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, matthews_corrcoef

initial_logreg = LogisticRegression(penalty='none', solver='newton-cg')
initial_logreg.fit(initial_X, initial_y)

initial_y_pred = initial_logreg.predict(X_test)

print(classification_report(y_test, initial_y_pred))
print(matthews_corrcoef(y_test, initial_y_pred))


In [ ]:
sorted([*zip(initial_X_test.columns, initial_logreg.coef_[0])], key = lambda pair: abs(pair[1]), reverse = True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, matthews_corrcoef

X = train.drop('left', axis=1)
y = train['left']

logreg = LogisticRegression(penalty='none', solver='newton-cg')
logreg.fit(X, y)

X_test = test.drop('left', axis=1)
y_test = test['left']

y_pred = logreg.predict(X_test)

print(classification_report(y_test, y_pred))
print(matthews_corrcoef(y_test, y_pred))


In [ ]:
sorted([*zip(X_test.columns, logreg.coef_[0])], key = lambda pair: abs(pair[1]), reverse = True)

#### Feature Engineering - regularized logit, all features

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, matthews_corrcoef

initial_logreg_l1 = LogisticRegression(penalty='l1', solver='liblinear')
initial_logreg_l1.fit(initial_X, initial_y)

initial_y_pred_l1 = initial_logreg_l1.predict(initial_X_test)

print(classification_report(initial_y_test, initial_y_pred_l1))
print(matthews_corrcoef(initial_y_test, initial_y_pred_l1))

In [ ]:
sorted([*zip(X_test.columns, initial_logreg_l1.coef_[0])], key = lambda pair: abs(pair[1]), reverse = True)